[View in Colaboratory](https://colab.research.google.com/github/Robert-Alonso/Kaggle-Titanic/blob/master/DisRegGen_CIFAR10_Final.ipynb)

# <center>Discriminative Regularize Generative Model for CIFAR10 </center>

## Load Data

In [1]:
!pip3 install --upgrade torch torchvision

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable

import torchvision
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
from IPython.display import Image
from google.colab import files

#Set random seed 
torch.manual_seed(512)

    100% |████████████████████████████████| 484.0MB 24kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x5cedc000 @  0x7fcedd4d51c4 0x46d6a4 0x5fcbcc 0x4c494d 0x54f3c4 0x553aaf 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54e4c8
    100% |████████████████████████████████| 61kB 20.5MB/s 
Requirement not upgraded as not directly required: six in /usr/local/lib/python3.6/dist-packages (from torchvision) (1.11.0)
    100% |████████████████████████████████| 2.0MB 11.6MB/s 
Requirement not upgraded as not directly required: numpy in /usr/local/lib/python3.6/dist-packages (from torchvision) (1.14.3)
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


In [0]:
class preTrainedModel(nn.Module):
  
    def __init__(self):
      
      super(preTrainedModel,self).__init__()
      
      vgg_model = torchvision.models.vgg16(pretrained=True)		
      self.Conv1 = nn.Sequential(*list(vgg_model.features.children())[0:4])
      self.Conv2 = nn.Sequential(*list(vgg_model.features.children())[4:9]) 
      #self.Conv3 = nn.Sequential(*list(vgg_model.features.children())[9:16])
      #self.upSample1 = nn.Upsample(scale_factor=2)
      #self.upSample2 = nn.Upsample(scale_factor=4)
      
      for param in self.parameters():
        param.requires_grad = False

    def forward(self,x):
      out1 = self.Conv1(x)
      out2 = self.Conv2(out1)
      #out3 = self.Conv3(out2)
      ###### up sampling to create output with the same size
      #out2 = self.upSample1(out2)
      #out3 = self.upSample2(out3)
      #concat_features = torch.cat([out1, out2, out3], 1)
      return (out1,out2)



In [0]:
class Vgg16(nn.Module):
  
    def __init__(self):
        super(Vgg16, self).__init__()
        features = torchvision.models.vgg16(pretrained=True).features
        self.to_relu_1_2 = nn.Sequential() 
        self.to_relu_2_2 = nn.Sequential() 
        self.to_relu_3_3 = nn.Sequential()
        self.to_relu_4_3 = nn.Sequential()

        for x in range(4):
            self.to_relu_1_2.add_module(str(x), features[x])
        for x in range(4, 9):
            self.to_relu_2_2.add_module(str(x), features[x])
        for x in range(9, 16):
            self.to_relu_3_3.add_module(str(x), features[x])
        for x in range(16, 23):
            self.to_relu_4_3.add_module(str(x), features[x])
        
        # don't need the gradients, just want the features
        for param in self.parameters():
            param.requires_grad = False

    def forward(self, x):
        h = self.to_relu_1_2(x)
        h_relu_1_2 = h
        h = self.to_relu_2_2(h)
        h_relu_2_2 = h
        h = self.to_relu_3_3(h)
        h_relu_3_3 = h
        h = self.to_relu_4_3(h)
        h_relu_4_3 = h
        out = (h_relu_1_2, h_relu_2_2, h_relu_3_3, h_relu_4_3)
        return out

In [0]:
#Load model 
vgg16 = preTrainedModel().eval().cuda()
#vgg16 = Vgg16().eval().cuda()

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /content/.torch/models/vgg16-397923af.pth
1.2%

28.5%

In [15]:
#Get the CIFAR10 train images 
cifar = datasets.CIFAR10('./data/cifar/', train = True, download = True)

# Organize training data in batches, 
# normalize them to have values between [-1, 1] (?)

train_images = torch.utils.data.DataLoader ( datasets.CIFAR10('./data/cifar/', train = True, download=False,
                               transform=transforms.Compose([
                               #transforms.Resize(64), 
                               #transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                               transforms.ToTensor(),])) , 
                               batch_size = 4, shuffle = True)

Files already downloaded and verified


In [16]:
upsampling = nn.Upsample(size=256)
for batch_idx, (data,_) in enumerate(train_images):    
    out = vgg16(upsampling(data.cuda()))
    
    print('Data size: ', data.size())
    print('Output 1 size: ', out[0].size())
    print('Output 2 size: ', out[1].size())
    #print('Output 3 size: ', out[2].size())
    #print('Output 4 size: ', out[3].size())
    
    break

Data size:  torch.Size([4, 3, 32, 32])
Output 1 size:  torch.Size([64, 256, 256])
Output 2 size:  torch.Size([64, 256, 256])


## Model

We will use the arquitecture suggested by [Radford et al](https://arxiv.org/abs/1511.06434) for both the encoder and decoder. With convolutional layers in the encoder and fractionally-strided  convolutions  in  the  decoder.   In  each convolutional layer in the encoder we double the number of filters present in the previous layer and use a convolutional stride of 2.  In each convolutional layer in the decoder we use a fractional stride of 2 and halve the number of filters on each layer.

In [0]:
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable

class VAE( nn.Module ):

    def __init__ ( self, image_size ,  hidden_dim , encoding_dim ):
        
        super( VAE, self ).__init__()
        
        self.encoding_dim = encoding_dim
        self.image_size = image_size
        self.hidden_dim = hidden_dim 
        
        # Decoder - Fractional strided convolutional layers
        self.decoder  = nn.Sequential(
            nn.ConvTranspose2d(256, 128, 4, 1, 0, bias = False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias = False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, 32, 4, 2, 1, bias = False),
            nn.BatchNorm2d(32),
            nn.ReLU(True),
            nn.ConvTranspose2d(32, 3, 4, 2, 1, bias = False),
            nn.Sigmoid() # nn.Tanh()  
        )
        
        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, 4, 2, 1, bias = False),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(32, 64, 4, 2, 1, bias = False),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(64, 128, 4, 2, 1, bias = False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(128, 256, 4, 2, 0, bias = False),
            nn.Sigmoid()
        )
        
        # Fully-connected layers
        self.fc1 = nn.Linear(256, self.hidden_dim)
        self.fc21 = nn.Linear(self.hidden_dim, self.encoding_dim)
        self.fc22 = nn.Linear(self.hidden_dim, self.encoding_dim)
        self.fc3 = nn.Linear(self.encoding_dim, self.hidden_dim)
        self.fc4 = nn.Linear(self.hidden_dim, 256)
    
    def decode (self, z):
        h3 = F.relu(self.fc3(z))
        h4 = F.sigmoid(self.fc4(h3))
        return self.decoder( h4.view(z.size(0),-1,1,1) ) 

        
    def forward(self, x):
        
        # Encode 
        encoded = F.relu(self.fc1( self.encoder(x).view(x.size(0), -1) ) )
        
        #Obtain mu and logvar
        mu = self.fc21( encoded )
        logvar = self.fc22 ( encoded )
        
        #Reparametrization trick
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        z = eps.mul(std).add_(mu)
        
        # Decode 
        decoded = self.decode(z)

        # return decoded, mu, logvar
        return decoded, mu , logvar


upsampling = nn.Upsample(size=256)
sigmoid = nn.Sigmoid()

# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar):
  
  # VAE LOSS
    BCE = F.binary_cross_entropy(recon_x, x, size_average=False)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
  
  # PERCEPTUAL LOSS
    recon_x_vgg16 = vgg16 ( upsampling( recon_x ) )
    x_vgg16 = vgg16 ( upsampling( x ) )
    
    L0 = F.mse_loss( Srecon_x_vgg16[0] , x_vgg16[0], size_average=False)
    L1 = F.mse_loss( recon_x_vgg16[1] , x_vgg16[1], size_average=False)
    #L2 = F.mse_loss( recon_x_vgg16[2] , x_vgg16[2], size_average=False)
    #L3 = F.mse_loss( recon_x_vgg16[3] , x_vgg16[3], size_average=False)
    #print('L0: ',L0.data)
    #print('L1:' ,L1.data)
    #print('L2: ',L2.data)
    #print('L3: ', L3.data)
    #print(BCE.data)
    #print(KLD.data)
    
    del recon_x_vgg16 
    del x_vgg16
    
    return BCE + KLD + L0 + L1

In [0]:
#Define model
model = VAE( 32, 100, 20 ).cuda()
#model.load_state_dict(torch.load('save_checkpoint_epoch_9.pth'))
  
  
optimizer = optim.Adam(model.parameters(), lr=1e-3)

#Train model
def train(epoch):
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_images):
        data = Variable(data).cuda()
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % 50 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_images.dataset),
                100. * batch_idx / len(train_images),
                loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_images.dataset)))

In [21]:
num_epochs = 5
for epoch in range(1,num_epochs):
    train(epoch)    

Train Epoch: 1 [0/50000 (0%)]	Loss: 116313.093750
Train Epoch: 1 [200/50000 (0%)]	Loss: 177329.093750
Train Epoch: 1 [400/50000 (1%)]	Loss: 90363.796875
Train Epoch: 1 [600/50000 (1%)]	Loss: 88329.031250
Train Epoch: 1 [800/50000 (2%)]	Loss: 111293.375000
Train Epoch: 1 [1000/50000 (2%)]	Loss: 76695.054688
Train Epoch: 1 [1200/50000 (2%)]	Loss: 139406.093750
Train Epoch: 1 [1400/50000 (3%)]	Loss: 85086.632812
Train Epoch: 1 [1600/50000 (3%)]	Loss: 55279.351562
Train Epoch: 1 [1800/50000 (4%)]	Loss: 86910.640625
Train Epoch: 1 [2000/50000 (4%)]	Loss: 65911.195312
Train Epoch: 1 [2200/50000 (4%)]	Loss: 133015.531250
Train Epoch: 1 [2400/50000 (5%)]	Loss: 77796.062500
Train Epoch: 1 [2600/50000 (5%)]	Loss: 92703.843750
Train Epoch: 1 [2800/50000 (6%)]	Loss: 97212.656250
Train Epoch: 1 [3000/50000 (6%)]	Loss: 122148.476562
Train Epoch: 1 [3200/50000 (6%)]	Loss: 133246.968750
Train Epoch: 1 [3400/50000 (7%)]	Loss: 73505.500000
Train Epoch: 1 [3600/50000 (7%)]	Loss: 64181.964844
Train Epoch:

Train Epoch: 1 [12200/50000 (24%)]	Loss: 107481.109375
Train Epoch: 1 [12400/50000 (25%)]	Loss: 107772.062500
Train Epoch: 1 [12600/50000 (25%)]	Loss: 166984.593750
Train Epoch: 1 [12800/50000 (26%)]	Loss: 135839.656250
Train Epoch: 1 [13000/50000 (26%)]	Loss: 97730.218750
Train Epoch: 1 [13200/50000 (26%)]	Loss: 75957.367188
Train Epoch: 1 [13400/50000 (27%)]	Loss: 107419.296875
Train Epoch: 1 [13600/50000 (27%)]	Loss: 81190.890625
Train Epoch: 1 [13800/50000 (28%)]	Loss: 89178.382812
Train Epoch: 1 [14000/50000 (28%)]	Loss: 81774.093750
Train Epoch: 1 [14200/50000 (28%)]	Loss: 131574.390625
Train Epoch: 1 [14400/50000 (29%)]	Loss: 135029.921875
Train Epoch: 1 [14600/50000 (29%)]	Loss: 95796.796875
Train Epoch: 1 [14800/50000 (30%)]	Loss: 140830.687500
Train Epoch: 1 [15000/50000 (30%)]	Loss: 107763.375000
Train Epoch: 1 [15200/50000 (30%)]	Loss: 127720.281250
Train Epoch: 1 [15400/50000 (31%)]	Loss: 94280.796875
Train Epoch: 1 [15600/50000 (31%)]	Loss: 87144.937500
Train Epoch: 1 [15

Train Epoch: 1 [24200/50000 (48%)]	Loss: 125420.453125
Train Epoch: 1 [24400/50000 (49%)]	Loss: 95299.570312
Train Epoch: 1 [24600/50000 (49%)]	Loss: 102346.015625
Train Epoch: 1 [24800/50000 (50%)]	Loss: 107715.507812
Train Epoch: 1 [25000/50000 (50%)]	Loss: 93819.921875
Train Epoch: 1 [25200/50000 (50%)]	Loss: 135878.765625
Train Epoch: 1 [25400/50000 (51%)]	Loss: 141209.437500
Train Epoch: 1 [25600/50000 (51%)]	Loss: 103624.539062
Train Epoch: 1 [25800/50000 (52%)]	Loss: 76942.562500
Train Epoch: 1 [26000/50000 (52%)]	Loss: 91740.523438
Train Epoch: 1 [26200/50000 (52%)]	Loss: 143995.593750
Train Epoch: 1 [26400/50000 (53%)]	Loss: 82297.437500
Train Epoch: 1 [26600/50000 (53%)]	Loss: 110785.898438
Train Epoch: 1 [26800/50000 (54%)]	Loss: 81875.210938
Train Epoch: 1 [27000/50000 (54%)]	Loss: 71414.007812
Train Epoch: 1 [27200/50000 (54%)]	Loss: 71028.804688
Train Epoch: 1 [27400/50000 (55%)]	Loss: 115013.929688
Train Epoch: 1 [27600/50000 (55%)]	Loss: 93134.234375
Train Epoch: 1 [278

Train Epoch: 1 [36200/50000 (72%)]	Loss: 89195.257812
Train Epoch: 1 [36400/50000 (73%)]	Loss: 89470.140625
Train Epoch: 1 [36600/50000 (73%)]	Loss: 138879.531250
Train Epoch: 1 [36800/50000 (74%)]	Loss: 108383.937500
Train Epoch: 1 [37000/50000 (74%)]	Loss: 94111.523438
Train Epoch: 1 [37200/50000 (74%)]	Loss: 86102.828125
Train Epoch: 1 [37400/50000 (75%)]	Loss: 76752.507812
Train Epoch: 1 [37600/50000 (75%)]	Loss: 91309.976562
Train Epoch: 1 [37800/50000 (76%)]	Loss: 49155.527344
Train Epoch: 1 [38000/50000 (76%)]	Loss: 148911.562500
Train Epoch: 1 [38200/50000 (76%)]	Loss: 97301.703125
Train Epoch: 1 [38400/50000 (77%)]	Loss: 88891.078125
Train Epoch: 1 [38600/50000 (77%)]	Loss: 160911.015625
Train Epoch: 1 [38800/50000 (78%)]	Loss: 79497.078125
Train Epoch: 1 [39000/50000 (78%)]	Loss: 96868.234375
Train Epoch: 1 [39200/50000 (78%)]	Loss: 207569.593750
Train Epoch: 1 [39400/50000 (79%)]	Loss: 59324.382812
Train Epoch: 1 [39600/50000 (79%)]	Loss: 78708.945312
Train Epoch: 1 [39800/5

Train Epoch: 1 [48200/50000 (96%)]	Loss: 77180.015625
Train Epoch: 1 [48400/50000 (97%)]	Loss: 46507.210938
Train Epoch: 1 [48600/50000 (97%)]	Loss: 114034.906250
Train Epoch: 1 [48800/50000 (98%)]	Loss: 64741.429688
Train Epoch: 1 [49000/50000 (98%)]	Loss: 106623.375000
Train Epoch: 1 [49200/50000 (98%)]	Loss: 128692.343750
Train Epoch: 1 [49400/50000 (99%)]	Loss: 91043.085938
Train Epoch: 1 [49600/50000 (99%)]	Loss: 92764.007812
Train Epoch: 1 [49800/50000 (100%)]	Loss: 57108.613281
====> Epoch: 1 Average loss: 96331.6684
Train Epoch: 2 [0/50000 (0%)]	Loss: 91829.179688
Train Epoch: 2 [200/50000 (0%)]	Loss: 74270.304688
Train Epoch: 2 [400/50000 (1%)]	Loss: 84665.570312
Train Epoch: 2 [600/50000 (1%)]	Loss: 98339.796875
Train Epoch: 2 [800/50000 (2%)]	Loss: 84191.031250
Train Epoch: 2 [1000/50000 (2%)]	Loss: 95502.359375
Train Epoch: 2 [1200/50000 (2%)]	Loss: 73450.984375
Train Epoch: 2 [1400/50000 (3%)]	Loss: 81611.812500
Train Epoch: 2 [1600/50000 (3%)]	Loss: 81013.109375
Train Epo

Train Epoch: 2 [10200/50000 (20%)]	Loss: 137283.453125
Train Epoch: 2 [10400/50000 (21%)]	Loss: 81030.531250
Train Epoch: 2 [10600/50000 (21%)]	Loss: 84680.968750
Train Epoch: 2 [10800/50000 (22%)]	Loss: 81813.000000
Train Epoch: 2 [11000/50000 (22%)]	Loss: 47717.761719
Train Epoch: 2 [11200/50000 (22%)]	Loss: 126276.984375
Train Epoch: 2 [11400/50000 (23%)]	Loss: 132691.578125
Train Epoch: 2 [11600/50000 (23%)]	Loss: 146149.375000
Train Epoch: 2 [11800/50000 (24%)]	Loss: 96342.937500
Train Epoch: 2 [12000/50000 (24%)]	Loss: 96670.000000
Train Epoch: 2 [12200/50000 (24%)]	Loss: 78887.460938
Train Epoch: 2 [12400/50000 (25%)]	Loss: 167553.343750
Train Epoch: 2 [12600/50000 (25%)]	Loss: 88895.687500
Train Epoch: 2 [12800/50000 (26%)]	Loss: 86295.585938
Train Epoch: 2 [13000/50000 (26%)]	Loss: 90745.718750
Train Epoch: 2 [13200/50000 (26%)]	Loss: 88119.125000
Train Epoch: 2 [13400/50000 (27%)]	Loss: 66335.500000
Train Epoch: 2 [13600/50000 (27%)]	Loss: 143227.484375
Train Epoch: 2 [13800/

Train Epoch: 2 [22200/50000 (44%)]	Loss: 71285.976562
Train Epoch: 2 [22400/50000 (45%)]	Loss: 94303.609375
Train Epoch: 2 [22600/50000 (45%)]	Loss: 103573.703125
Train Epoch: 2 [22800/50000 (46%)]	Loss: 102350.781250
Train Epoch: 2 [23000/50000 (46%)]	Loss: 108690.851562
Train Epoch: 2 [23200/50000 (46%)]	Loss: 52095.976562
Train Epoch: 2 [23400/50000 (47%)]	Loss: 129563.859375
Train Epoch: 2 [23600/50000 (47%)]	Loss: 81429.625000
Train Epoch: 2 [23800/50000 (48%)]	Loss: 78631.335938
Train Epoch: 2 [24000/50000 (48%)]	Loss: 79730.859375
Train Epoch: 2 [24200/50000 (48%)]	Loss: 117729.445312
Train Epoch: 2 [24400/50000 (49%)]	Loss: 95491.406250
Train Epoch: 2 [24600/50000 (49%)]	Loss: 104785.890625
Train Epoch: 2 [24800/50000 (50%)]	Loss: 129496.179688
Train Epoch: 2 [25000/50000 (50%)]	Loss: 50710.976562
Train Epoch: 2 [25200/50000 (50%)]	Loss: 113850.187500
Train Epoch: 2 [25400/50000 (51%)]	Loss: 91233.015625
Train Epoch: 2 [25600/50000 (51%)]	Loss: 77884.664062
Train Epoch: 2 [2580

Train Epoch: 2 [34200/50000 (68%)]	Loss: 109279.820312
Train Epoch: 2 [34400/50000 (69%)]	Loss: 70319.015625
Train Epoch: 2 [34600/50000 (69%)]	Loss: 120165.421875
Train Epoch: 2 [34800/50000 (70%)]	Loss: 93533.656250
Train Epoch: 2 [35000/50000 (70%)]	Loss: 132103.109375
Train Epoch: 2 [35200/50000 (70%)]	Loss: 52153.328125
Train Epoch: 2 [35400/50000 (71%)]	Loss: 92125.265625
Train Epoch: 2 [35600/50000 (71%)]	Loss: 122928.484375
Train Epoch: 2 [35800/50000 (72%)]	Loss: 105314.484375
Train Epoch: 2 [36000/50000 (72%)]	Loss: 89842.781250
Train Epoch: 2 [36200/50000 (72%)]	Loss: 71909.585938
Train Epoch: 2 [36400/50000 (73%)]	Loss: 77231.414062
Train Epoch: 2 [36600/50000 (73%)]	Loss: 110416.062500
Train Epoch: 2 [36800/50000 (74%)]	Loss: 78321.656250
Train Epoch: 2 [37000/50000 (74%)]	Loss: 60457.781250
Train Epoch: 2 [37200/50000 (74%)]	Loss: 79329.757812
Train Epoch: 2 [37400/50000 (75%)]	Loss: 59211.546875
Train Epoch: 2 [37600/50000 (75%)]	Loss: 62300.406250
Train Epoch: 2 [37800/

Train Epoch: 2 [46200/50000 (92%)]	Loss: 88385.648438
Train Epoch: 2 [46400/50000 (93%)]	Loss: 83192.679688
Train Epoch: 2 [46600/50000 (93%)]	Loss: 80808.578125
Train Epoch: 2 [46800/50000 (94%)]	Loss: 75559.656250
Train Epoch: 2 [47000/50000 (94%)]	Loss: 145251.109375
Train Epoch: 2 [47200/50000 (94%)]	Loss: 76239.218750
Train Epoch: 2 [47400/50000 (95%)]	Loss: 70946.750000
Train Epoch: 2 [47600/50000 (95%)]	Loss: 69033.109375
Train Epoch: 2 [47800/50000 (96%)]	Loss: 63721.648438
Train Epoch: 2 [48000/50000 (96%)]	Loss: 56590.191406
Train Epoch: 2 [48200/50000 (96%)]	Loss: 74576.726562
Train Epoch: 2 [48400/50000 (97%)]	Loss: 117995.812500
Train Epoch: 2 [48600/50000 (97%)]	Loss: 75117.804688
Train Epoch: 2 [48800/50000 (98%)]	Loss: 98094.875000
Train Epoch: 2 [49000/50000 (98%)]	Loss: 94268.101562
Train Epoch: 2 [49200/50000 (98%)]	Loss: 94207.195312
Train Epoch: 2 [49400/50000 (99%)]	Loss: 47726.941406
Train Epoch: 2 [49600/50000 (99%)]	Loss: 124553.468750
Train Epoch: 2 [49800/500

KeyboardInterrupt: ignored

In [13]:
with torch.no_grad():
        sample = torch.randn(64, 20).cuda()
        sample = model.decode(sample)
        torchvision.utils.save_image(sample.view(64, 3, 32, 32),'sample_' + str(epoch) + '.png')
        files.download('sample_' + str(epoch) + '.png')

AttributeError: ignored

In [0]:

torch.save(model.cpu().state_dict(), "save_checkpoint_epoch_"+str(epoch)+".pth")
files.download("save_checkpoint_epoch_"+str(epoch)+".pth")

In [22]:
!pip install Pillow==4.0.0
!pip install PIL
!pip install image

sample = torch.randn(64, 20).cuda()
sample = model.decode(sample)
%matplotlib inline
torchvision.utils.save_image(sample.view(64, 3, 32, 32),'sample_' + str(epoch) + '.png')
files.download('sample_' + str(epoch) + '.png')

  Using cached https://files.pythonhosted.org/packages/37/e8/b3fbf87b0188d22246678f8cd61e23e31caa1769ebc06f1664e2e5fe8a17/Pillow-4.0.0-cp36-cp36m-manylinux1_x86_64.whl
torchvision 0.2.1 has requirement pillow>=4.1.1, but you'll have pillow 4.0.0 which is incompatible.
  Found existing installation: Pillow 5.1.0
    Uninstalling Pillow-5.1.0:
      Successfully uninstalled Pillow-5.1.0
  Could not find a version that satisfies the requirement PIL (from versions: )
No matching distribution found for PIL
torchvision 0.2.1 has requirement pillow>=4.1.1, but you'll have pillow 4.0.0 which is incompatible.
